# 1. Data Collection

### 1.1 Get the List of Animes

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import urllib.request 
import os
import codecs
import pandas as pd
from datetime import datetime
import time
import csv

Downloading each anime's url to a single text file

In [24]:
links_text = open("links.txt", "w")
for page in tqdm(range(0, 400)):
    url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:        
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                data = link.get('href')
                links_text.write(data)
                links_text.write("\n")
links_text.close()

100%|██████████| 400/400 [05:44<00:00,  1.16it/s]


Reading how many lines in the text file

In [25]:
file = open("links.txt", "r")
line_count = 0
for line in file:
    if line != "\n":
        line_count += 1
file.close()

print('There are total {} lines in this file.'.format(line_count))


There are total 19124 lines in this file.


### 1.2 Crawl the Animes

In [161]:
directory = 'html_pages'
parent_dir = "/Users/onurergun/Desktop/ADM-HW3"
path = os.path.join(parent_dir, directory)
os.mkdir(path)

Creating subfolders for each page of the list </br>
After creating the subfolders, we add each html file to corresponding page number folder.

In [7]:
directory = 'html_pages'
file_read = open('links.txt', 'r')
anime_urls_list = file_read.readlines()
file_read.close()

for i in range(1,401):
    html_page_name = 'page'+str(i)
    os.makedirs(os.path.join(directory, html_page_name ))
    directory_subfolder = directory+'/'+html_page_name+'/'
    
    for j in range(1,51):
        anime_num = 50*(i-1)+j
        html_file_name = directory_subfolder+'article_'+str(anime_num)+'.html'
        temp_text = open(html_file_name, "w")
        url = anime_urls_list[(anime_num-1)].encode('ascii','backslashreplace').decode('utf-8')
        urllib.request.urlretrieve(url,html_file_name)
        temp_text.close()

FileExistsError: [Errno 17] File exists: 'html_pages/page1'

### 1.3 Parse downloaded pages


In [5]:
animeTitle = []
animeType = []
animeNumEpisode = []
releaseDate = []
endDate = []
animeNumMembers = []
animeScore = []
animeUsers = []
animeRank = []
animePopularity = []
animeDescription = []
animeRelated = []
animeCharacters = []
animeVoices = []
animeStaff = []
directory = 'html_pages'

In [6]:
def parse_function(html_file_path):
    """
    Function that extracts anime's informations.
    Input: path (a string that is related to the position of each anime page in the folder tree)
    Output: a list of lists with all the informations mentioned above
    """
    # take article_i.html from the directory 
    soup = BeautifulSoup(open(html_file_path), "html.parser")
    divs = soup.find_all("div", {"class": "spaceit_pad"})
    try:
        animeTitle.append(str(soup.find_all('strong')[0].contents[0]))
    except:
        animeTitle.append('')

    for div in divs:
        spans = div.find_all("span")
        for span in spans:
            # TYPES
            if span.contents[0] == 'Type:':
                try:
                    animeType.append(str(div.find_all('a')[0].contents[0]))
                except:
                    animeType.append('NA')
            # NUMBER OF EPISODES
            if span.contents[0] == 'Episodes:':
                try: 
                    animeNumEpisode.append(int(div.contents[2]))
                except:
                    animeNumEpisode.append(0)
            # DATES
            if span.contents[0] == 'Aired:':
                try:
                    if len(div.contents[2]) > 21:
                        release = pd.to_datetime(div.contents[2][1:16]).to_pydatetime().strftime('%m/%d/%Y')
                        releaseDate.append(release)
                        end = pd.to_datetime(div.contents[2][1:16]).to_pydatetime().strftime('%m/%d/%Y')
                        endDate.append(end)
                    else:
                        release = pd.to_datetime(div.contents[2][1:16]).to_pydatetime().strftime('%m/%d/%Y')
                        releaseDate.append(release)
                        endDate.append('-')
                except:
                        releaseDate.append('')
                        endDate.append('')

    divs = soup.find_all("div", {"class": "stats-block po-r clearfix"})
    for div in divs:
        
        # MEMBERS
        members = div.find_all("span", {"class": "numbers members"})
        animeNumMembers.append(int(members[0].contents[1].contents[0].replace(',', '')))
        
        
        # SCORE
        # center of the html page
        rating=soup.find(name="div",attrs={"class":"fl-l score"})
        try:        
            animeScore.append(float(rating.text.strip()))
        except:
            animeScore.append(None)

     
        # USERS
        users = div.find_all("div", {"class": "fl-l score"})
        # here we we eliminate the word 'user '   
        # that is why there is the [:-6] part
        # we also replace the comma divisor
        try:
            animeUsers.append(int(users[0]['data-user'][:-6].replace(',', '')))
        except:
            animeUsers.append(0)


        # RANK
        rank = div.find_all("span", {"class": "numbers ranked"})
        try:
            animeRank.append(int(rank[0].contents[1].contents[0][1:]))
        except:
            animeRank.append(None)

        # POPULARITY
        popularity = div.find_all("span", {"class": "numbers popularity"})
        animePopularity.append(int(popularity[0].contents[1].contents[0][1:]))
    
    # DESCRIPTION
    # center of the html page
    animeDescription.append(soup.find_all("p", itemprop = "description")[0].text.strip().replace('\n', '').replace('  ', ''))


    # RELATED 
    related = soup.find_all("table", {"class": "anime_detail_related_anime"})
    if(len(related)!=0):
        x = []
        y = []
        for tr in related:
            td = tr.find_all("td")
            for i in range(0, len(td), 2):
                x.append(td[i].contents[0])
                t = td[i+1].find_all("a")
                if(len(t[0].contents)!=0):  
                    y.append(t[0].contents[0])
                else:
                    y.append(' ')
            animeRelated.append('\n'.join([f'{x} {y}' for x, y in dict(zip(x, y)).items()]).split('\n'))
    else:
        animeRelated.append(' ')
    
    # CHARACTERS
    try:
        characters = soup.find_all("div", {"class": "detail-characters-list clearfix"})
        chars = characters[0].find_all("h3", {"class": "h3_characters_voice_actors"})
        x = []
        for i in chars:
            x.append(i.contents[0].contents[0])
        animeCharacters.append(x)
    except:
        animeCharacters.append(" ")
    
    
   # VOICES
    try:
        voices = characters[0].find_all("td", {"class": "va-t ar pl4 pr4"})
        y = []
        for i in voices:
            y.append(i.contents[1].contents[0])
        animeVoices.append(y)
    except:
        animeVoices.append(" ")
    
    # STAFF
    try:
        staff = soup.find_all("div", {"class": "detail-characters-list clearfix"})
        staff = staff[1].find_all("td")
        x = []
        y = []
        for i in range(1, len(staff), 2):
            x.append(staff[i].contents[1].contents[0])
            y.append(staff[i].find_all("small")[0].contents[0])
        animeStaff.append([list(i) for i in list(zip(x,y))])
    
    except:
        animeStaff.append(" ")
               

In [7]:
os.makedirs('tsv_files')

In [8]:
def tsv_create(i):
    tsv_columns = ['animeTitle','animeType','animeNumEpisode','releaseDate','endDate','animeNumMembers','animeScore',
                  'animeUsers','animeRank','animePopularity','animeDescription','animeRelated','animeCharacters',
                  'animeVoices','animeStaff']
    data = zip([animeTitle[i-1]],[animeType[i-1]],[animeNumEpisode[i-1]],[releaseDate[i-1]],[endDate[i-1]],[animeNumMembers[i-1]],[animeScore[i-1]],[animeUsers[i-1]],[animeRank[i-1]],[animePopularity[i-1]],[animeDescription[i-1]],[animeRelated[i-1]],[animeCharacters[i-1]],[animeVoices[i-1]],[animeStaff[i-1]])
    tsv_file_name = 'tsv_files/anime_'+str(i)+'.tsv'
    with open(tsv_file_name, 'w', newline='') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        tsv_output.writerow(tsv_columns)
        for title,typ,numEp,relD,endD,numMem,score,user,rank,popularity,descr,relat,charac,voices,staff in data:
                tsv_output.writerow([title,typ,numEp,relD,endD,numMem,score,user,rank,popularity,descr,relat,charac,voices,staff])


In [9]:
for i in range(1,384):
    html_page_name = 'page'+str(i)
    directory_subfolder = directory+'/'+html_page_name+'/'
    if(i!=383):
        # 383th page has less than 50 animes
        for j in range(1,51):
            anime_num = 50*(i-1)+j
            html_file_path = directory_subfolder+'article_'+str(anime_num)+'.html'
            soup = BeautifulSoup(open(html_file_path), "html.parser")
            parse_function(html_file_path)
            tsv_create(anime_num)
    else:
        for j in range(1,25):
            anime_num = 50*(i-1)+j
            html_file_path = directory_subfolder+'article_'+str(anime_num)+'.html'
            soup = BeautifulSoup(open(html_file_path), "html.parser")
            parse_function(html_file_path)
            tsv_create(anime_num)

# 2. Search Engine


### Preprocessing of the information

In [2]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import numpy as np
import json

In [148]:
stop = stopwords.words('english')
porter_stemmer = PorterStemmer()

In [149]:
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)


In [150]:
animeTitle_list = []
animeDescription_list = []

In [151]:
# creating a lists from tsv files
for i in range(0,19124):
    anime_tsv = open('tsv_files/anime_'+str(i+1)+'.tsv', 'r',encoding="utf8")
    data=pd.read_table(anime_tsv)[['animeTitle','animeDescription']]
    data['animeTitle'] = data['animeTitle'].astype(str)
    data['animeDescription'] = data['animeDescription'].astype(str)
    animeTitle_list.append(str(data.animeTitle[0]))
    animeDescription_list.append(str(data.animeDescription[0]))

In [152]:
# concetenating lists to create a dataframe
anime_df = pd.DataFrame(np.column_stack([animeTitle_list, animeDescription_list]), 
                               columns=['animeTitle', 'animeDescription'])

Checking if there is a mismatch on the dataframe

In [153]:
# removing stopwords from the dataframe
anime_df['animeDescription']  = anime_df['animeDescription'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# removing punctuations from the dataframe
anime_df['animeDescription'] = anime_df['animeDescription'].str.replace('[^\w\s]',' ')

# stemming the dataframe 
anime_df['animeDescription'] = anime_df['animeDescription'].apply(stem_sentences)


<ipython-input-153-b8d327cbacaf>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  anime_df['animeDescription'] = anime_df['animeDescription'].str.replace('[^\w\s]',' ')


In [154]:
words_list = ' '.join([i for i in anime_df['animeDescription']]).split()

### 2.1. Conjunctive query


In [155]:
# removing the duplicate words
words_dict = set(words_list)

# Assign a unique integer id to each non duplicate words
vocabulary = {}
i=1
for word in words_dict:
    vocabulary.update({i:word})
    i+=1

#### 2.1.1 Create your index!


Creating the vocabulary json file that we store for each nun duplicate words with id number 

In [156]:
with open("vocabulary.json", "w") as file:
    json.dump(vocabulary, file)
file.close()

Creating an inverted index 

In [157]:
inverted_dict = {}

In [158]:
with open('vocabulary.json') as data_file:    
    data = json.load(data_file)
    for key, value in tqdm(data.items()):
        inverted_list = []
        for i in range(0,len(anime_df)):
            if(value in anime_df['animeDescription'][i].split()):
                anime_name = 'anime_'+str(i+1)
                inverted_list.append(anime_name)
                inverted_dict.update({key:inverted_list})


100%|██████████| 36793/36793 [1:17:26<00:00,  7.92it/s]   


Saving the inverted index dictionary to a json file

In [159]:
with open("inverted.json", "w") as file:
    json.dump(inverted_dict, file)
file.close()

#### 2.1.2 Execute the query


In [94]:
def find_query(query_list):
    anime_query_list = []
    for word in query_list:
        with open('vocabulary.json') as data_file:
            data = json.load(data_file)
            for key, value in data.items():
                if(word == value):
                    with open('inverted.json') as inverted_file:
                        inverted_data = json.load(inverted_file)
                        for inv_key, inv_value in inverted_data.items():
                            if(key == inv_key):
                                # appending the value to a list if has the specific query word
                                anime_query_list.append(inv_value)
    
    # creating a list from all animes including duplicate ones
    anime_list = []
    for i in range(len(anime_query_list)):
        for j in range(len(anime_query_list[i])):
            anime_list.append(anime_query_list[i][j])
    
    # creating a set to find non duplicate anime files
    anime_query_set_list = list(set(anime_list))
    # creating an empty list to store the final anime list which has all the input queries 
    anime_final_list = []
    
    # counting the occurences of each anime with the length of the total query
    # if its equal to total len, then each word in the query appears on the anime description
    for anime in anime_query_set_list:
        if(anime_list.count(anime) == len(anime_query_list)):
            anime_final_list.append(anime)
    return anime_final_list

In [132]:
def create_query_anime_df(anime_list):
    # creating lists for animes
    animeTitle_list = []
    animeDescription_list = []
    animeUrl_list = []

    # assigning tsv values from the animes to lists we've just created 
    for anime in anime_list:
        anime_tsv = open('tsv_files/'+anime+'.tsv', 'r',encoding="utf8")
        data=pd.read_table(anime_tsv)[['animeTitle','animeDescription']]
        data['animeTitle'] = data['animeTitle'].astype(str)
        data['animeDescription'] = data['animeDescription'].astype(str)
        animeTitle_list.append(str(data.animeTitle[0]))
        animeDescription_list.append(str(data.animeDescription[0]))

    # reading text file url lines to a list
    f=open('links.txt')
    url_lines=f.readlines()
    f.close()

    # creating a for loop to iterate over each anime we have on the anime_list
    for anime in anime_list:
        # getting the int value from the anime name
        anime_num=(int(anime.split("anime_",1)[1]))
        # finding the corresponding line from the links.txt and assigning it to a list
        animeUrl_list.append(url_lines[(anime_num-1)])

    # creating the dataframe from lists and returning it
    return pd.DataFrame(np.column_stack([animeTitle_list, animeDescription_list, animeUrl_list]), 
                                   columns=['animeTitle', 'animeDescription', 'Url'])

In [185]:
# getting an input from the user
query = input('Enter your search:')

Enter your search: Written by MAL Rewrite


In [186]:
# creating a list from the input query
query_list = query.split()
# getting the list of animes which has the query
anime_list = find_query(query_list)

In [187]:
# getting the anime dataframe from our query
query_anime_df = create_query_anime_df(anime_list)

In [188]:
query_anime_df

,animeTitle,animeDescription,Url
0,U.F.O,"Without using a camera, this film was made on ...",https://myanimelist.net/anime/38876/UFO\n
1,Shuumatsu Nani Shitemasu ka? Isogashii Desu ka...,"Putting his life on the line, Willem Kmetsch l...",https://myanimelist.net/anime/33502/Shuumatsu_...
2,AIKa Zero,"Aika's story continues, she is now 19 years-ol...",https://myanimelist.net/anime/6443/AIKa_Zero\n
3,Love Like Aliens,"In a future not too far from now, humanity has...",https://myanimelist.net/anime/17293/Love_Like_...
4,Mobile Suit Gundam Unicorn RE:0096,"By the year 0096 of the Universal Century, a f...",https://myanimelist.net/anime/32792/Mobile_Sui...
...,...,...,...
177,Aoki Hagane no Arpeggio: Ars Nova Movie 1 - DC,Recap of the Aoki Hagane no Arpeggio: Ars Nova...,https://myanimelist.net/anime/24919/Aoki_Hagan...
178,Initial D Extra Stage 2,Iketani missed an opportunity to see Mako-chan...,https://myanimelist.net/anime/5228/Initial_D_E...
179,Namakura Gatana,"Namakura Gatana, meaning ""dull-edged sword,"" i...",https://myanimelist.net/anime/6654/Namakura_Ga...
180,Choujin Locke: Lord Leon,It is the year 336 of the Space Century on the...,https://myanimelist.net/anime/4014/Choujin_Loc...


### 2.2 Conjunctive query & Ranking score


SyntaxError: invalid syntax (<ipython-input-172-b16655eb6892>, line 1)